In [1]:
!pip install accelerate -U
!pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 55.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.1
    Uninstalling tokenizers-0.15.1:
      Successfully uninstalled tokenizers-0.15.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
!pip install opendatasets


In [3]:
import pandas as pd

import torch
import torchvision

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import opendatasets as od

from PIL import Image
import os
import json
from transformers import AutoModel, AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel, BertConfig
import transformers

import accelerate
from torchvision import transforms
import torch.nn as nn
import torch
import csv


In [4]:
dataset_url = 'https://www.kaggle.com/datasets/trungit/wikiart25k'
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ruthleee
Your Kaggle Key: ··········


100%|██████████| 7.74G/7.74G [00:30<00:00, 275MB/s]


In [5]:
import os

# function to delete incorrectly encoded files

def delete_non_ascii_filenames(directory):
    count = 0
    for subdir, _, files in os.walk(directory):
        for file in files:
            try:
                file.encode('ascii')
            except UnicodeEncodeError:  # Filename is non-ASCII
                full_path = os.path.join(subdir, file)
                os.remove(full_path)  # Delete the file
                count += 1
                print(f"Deleted file: {full_path}")
    return count

# Use the function on your dataset directory and delete non-ASCII files
deleted_count = delete_non_ascii_filenames('wikiart25k')
print(f"Total deleted files: {deleted_count}")

Deleted file: wikiart25k/Baroque/mestre-ata├ú┬¡de_a-coroa-o-da-virgem-pela-sant-ssima-trindade-detail-1811.jpg
Deleted file: wikiart25k/Baroque/mestre-ata├ú┬¡de_ascens-o-de-cristo.jpg
Deleted file: wikiart25k/Baroque/mestre-ata├ú┬¡de_a-ltima-ceia-1828.jpg
Deleted file: wikiart25k/Baroque/mestre-ata├ú┬¡de_assun-o-da-virgem-detail-1823.jpg
Deleted file: wikiart25k/Symbolism/arnold-b├ú┬╢cklin_lament-of-the-shepherd-1866.jpg
Deleted file: wikiart25k/Symbolism/arnold-b├ú┬╢cklin_gottfried-keller.jpg
Deleted file: wikiart25k/Symbolism/arnold-b├ú┬╢cklin_the-rest-on-the-flight-into-egypt.jpg
Deleted file: wikiart25k/Symbolism/georges-lacombe├ó┬á_women-damned.jpg
Deleted file: wikiart25k/Symbolism/arnold-b├ú┬╢cklin_the-sea.jpg
Deleted file: wikiart25k/Symbolism/arnold-b├ú┬╢cklin_sirens-1875.jpg
Deleted file: wikiart25k/Symbolism/arnold-b├ú┬╢cklin_melancholia.jpg
Deleted file: wikiart25k/Symbolism/arnold-b├ú┬╢cklin_attack-by-pirates.jpg
Deleted file: wikiart25k/Symbolism/georges-lacombe├ó┬á_yello

In [6]:
ARTEMIS_EMOTIONS = ['amusement', 'awe', 'contentment', 'excitement',
                    'anger', 'disgust',  'fear', 'sadness', 'something else']

EMOTION_TO_IDX = {e: i for i, e in enumerate(ARTEMIS_EMOTIONS)}


IDX_TO_EMOTION = {EMOTION_TO_IDX[e]: e for e in EMOTION_TO_IDX}

In [7]:
class MoodBoardDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.img_annotations = self._load_annotations(csv_file)
        self.transform = transform

    def _load_annotations(self, csv_file):
        with open(csv_file, 'r', newline='') as file:
            reader = csv.reader(file)
            # Skip the header if there is one
            next(reader, None)
            img_annots = [row for row in reader]
        return img_annots

    def __len__(self):
        return len(self.img_annotations)

    def get_image_path(art_style, painting, top_dir='/wikiart25k'):
      path = f"{top_dir}/{art_style}/{painting}.jpg"
      print(path)
      return path

    def __getitem__(self, x):
        img_info = self.img_annotations[x]
        img_path = 'wikiart25k/' + img_info[0] + '/' + img_info[1] + '.jpg'
        image = Image.open(img_path).convert('RGB')
        emotion = img_info[2]
        utterance = img_info[3]

        if self.transform:
            image = self.transform(image)

        # Create the sample as a dictionary
        sample = {"image": image, "emotion": emotion, "utterance": utterance}
        return sample

In [8]:
tokenizer = AutoTokenizer.from_pretrained("ayoubkirouane/BERT-Emotions-Classifier")
def data_collate_fn(batch):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    images = [transform(sample['image']) for sample in batch]
    images = torch.stack(images, dim=0)

    label_mapping = {
        "amusement": 0,
        "awe": 1,
        "contentment": 2,
        "excitement": 3,
        "anger": 4,
        "disgust": 5,
        "fear": 6,
        "sadness": 7,
        "something else":8
    }
    emotion = [label_mapping[sample['emotion']] for sample in batch]
    labels = torch.tensor(emotion, dtype=torch.long).detach()

    texts = [sample["utterance"] for sample in batch]

    # Tokenized in the collate fn for convenience when used in model, directly passed into forward
    tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    return {"images": images, "input_ids": tokenized_texts['input_ids'], "labels": labels}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [10]:
data = pd.read_csv('trimmed_valid_images.csv')
train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)  # 20% for testing
train_set.to_csv('train_data.csv', index=False)
test_set.to_csv('test_data.csv', index=False)

In [11]:
# Dataset objects to feed into trainer
train_dataset = MoodBoardDataset(csv_file='train_data.csv')
test_dataset = MoodBoardDataset(csv_file='test_data.csv')

In [34]:
minidata = pd.read_csv('test_data.csv')
minitrain, minitests = train_test_split(minidata, test_size=0.3, random_state=42)
minitrain.to_csv('minitrain.csv', index=False)
minitests.to_csv('minitests.csv', index=False)

In [35]:
minitrain = MoodBoardDataset(csv_file='minitrain.csv')

minitests = MoodBoardDataset(csv_file='minitests.csv')

In [12]:
  class MultiModalEarlyFusionBertModel(BertModel):
    def __init__(self, config, num_classes, cnn_model):
        super().__init__(config)

        self.cnn_model = torchvision.models.resnet18(pretrained=True)
        # Created f_cnn because we want to run the images through the layers before linear layers
        self.f_cnn = torch.nn.Sequential(*list(resnet_model.children())[:-2])
        self.image_layer= nn.Linear(512, config.hidden_size)

        # Using a specific pretrained BertModel that was trained for test to emotion purposes with multiclassification
        self.bert_model = AutoModel.from_pretrained("ayoubkirouane/BERT-Emotions-Classifier", output_hidden_states=True)
        self.modality_embedding = nn.Embedding(num_classes, config.hidden_size)
        self.fusion_linear = nn.Linear(config.hidden_size, num_classes)
        self.loss_fct = nn.CrossEntropyLoss()



    def forward(self, input_ids=None, attention_mask=None, images=None, labels=None):
        image_embeddings = self.f_cnn(images)
        flat_image_embeddings = image_embeddings.view(image_embeddings.size(0), -1, image_embeddings.size(1))

        # Take image embeddings before the linear layer B, 7*7, 2000 and then flatten to 3d

        text_outputs = self.bert_model(input_ids=input_ids)
        # Hidden states are in the third element of the outputs tuple
        text_embeddings = text_outputs.hidden_states[-2]

        batch_size = image_embeddings.size(0)

        # Reshaped to make it compatible with our text embeddings
        reshaped_image_embedding = self.image_layer(flat_image_embeddings)

        image_modality_embeddings = self.modality_embedding(torch.zeros_like(reshaped_image_embedding[:, :, 0]).long().to(images[0].device))
        text_modality_embeddings = self.modality_embedding(torch.zeros_like(text_embeddings[:, :, 0]).long().to(input_ids[0].device))

        # FUSE image embeddings with modality embeddings, same for text
        fused_image_embeddings = reshaped_image_embedding + image_modality_embeddings

        text_size = text_embeddings.size(0)
        fused_text_embeddings = text_modality_embeddings + text_embeddings

        fused_image_embeddings = torch.reshape(fused_image_embeddings, (batch_size, -1, 768))
        fused_text_embeddings = torch.reshape(fused_text_embeddings, (batch_size, -1, 768))
        fused_embeddings = torch.cat((fused_image_embeddings, fused_text_embeddings), dim=1)
        pooled_embeddings, _ = torch.max(fused_embeddings, dim=1)

        # Makes it 2Dim gets rid of second dim (95)
        logits = self.fusion_linear(pooled_embeddings)
        # aggregated_logits = torch.mean(logits, dim=1)

        outputs = {'logits': logits}
        if labels is not None:
          loss = self.loss_fct(logits,labels.view(-1))
          outputs ['loss'] = loss
        return outputs

        # return(logits)

config = BertConfig.from_pretrained('bert-base-uncased')
resnet_model = torchvision.models.resnet18(pretrained=True)
multimodel = MultiModalEarlyFusionBertModel(config, num_classes=9, cnn_model=resnet_model)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 154MB/s]


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [13]:
from sklearn.metrics import accuracy_score
import numpy as np

# Calculates what percentage of labels were predicted correctly
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    accuracy = accuracy_score(p.label_ids, preds)
    print("Accuracy:", accuracy)
    return {"accuracy": accuracy}

In [15]:
training_args = TrainingArguments(
    num_train_epochs=10,
    learning_rate=5e-6,
    weight_decay=0.01,
    logging_dir='./logs',
    output_dir='/results',
    lr_scheduler_type="cosine",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    remove_unused_columns=False,
    load_best_model_at_end=True,
)

optimizer = torch.optim.AdamW(multimodel.parameters(), lr=training_args.learning_rate)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=500)

trainer = Trainer(
    model=multimodel,
    args=training_args,
    train_dataset=train_dataset,  # Specify your training dataset
    eval_dataset=test_dataset,
    # optimizers=(optimizer, scheduler),
    data_collator=data_collate_fn,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()
multimodel.save_pretrained('./best_model1')
tokenizer.save_pretrained('./best_model1')

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.298500,1.084984,0.626141
2,1.014400,1.035739,0.638151
3,0.932700,1.021741,0.642515
4,0.871300,1.012305,0.648105
5,0.823200,1.015903,0.649042
6,0.782300,1.016664,0.648826
7,0.753900,1.021297,0.648718
8,0.732600,1.022998,0.649547
9,0.721900,1.023710,0.650341
10,0.717300,1.024447,0.649583


[2 6 8 ... 0 2 5]
27727
[2 6 8 ... 3 2 8]
Accuracy: 0.6261405849893605
[2 6 8 ... 0 2 5]
27727
[2 6 8 ... 3 3 8]
Accuracy: 0.6381505391856314
[2 6 8 ... 0 2 5]
27727
[2 6 8 ... 2 3 7]
Accuracy: 0.6425145165362283
[2 6 8 ... 0 2 5]
27727
[2 6 8 ... 2 3 8]
Accuracy: 0.6481047354564143
[2 6 8 ... 0 2 5]
27727
[2 6 8 ... 2 3 8]
Accuracy: 0.649042449597865
[2 6 8 ... 0 2 5]
27727
[2 6 8 ... 2 3 8]
Accuracy: 0.648826054026761
[2 6 8 ... 0 2 5]
27727
[2 6 8 ... 2 3 7]
Accuracy: 0.6487178562412089
[2 6 8 ... 0 2 5]
27727
[2 6 8 ... 2 3 7]
Accuracy: 0.6495473725971075
[2 6 8 ... 0 2 5]
27727
[2 6 8 ... 2 3 7]
Accuracy: 0.6503408230244888
[2 6 8 ... 0 2 5]
27727
[2 6 8 ... 2 3 7]
Accuracy: 0.6495834385256248


[2 6 8 ... 0 2 5]
27727
[2 6 8 ... 2 3 8]
Accuracy: 0.6481047354564143


('./best_model1/tokenizer_config.json',
 './best_model1/special_tokens_map.json',
 './best_model1/vocab.txt',
 './best_model1/added_tokens.json',
 './best_model1/tokenizer.json')